<h1 style="font-family: 'Roboto', sans-serif; font-size: 32px; color: #008080;"> Agricultural Pest Classification using Deep Learning: ResNet and InceptionV3

![image.png](attachment:d2d24925-96dd-407b-8d02-f3db637d0023.png)
    
<h2 style="font-family: 'Roboto', sans-serif; font-size: 24px; color: #000000;">Introduction:</h2>

<p style="font-family: 'Arial', sans-serif; font-size: 18px; color: #333333;text-align: justify;">Welcome to this Kaggle notebook, where we explore the fascinating world of agricultural pest classification using the power of deep learning. In this project, we aim to assist farmers in effectively identifying and managing common agricultural pests using two powerful pre-trained deep learning models, ResNet and InceptionV3.

<p style="font-family: 'Arial', sans-serif; font-size: 18px; color: #333333;text-align: justify;">Agricultural pests pose significant threats to crop yield and quality, making their timely detection and appropriate management critical for sustainable agriculture. Traditional pest identification methods can be time-consuming and error-prone. Leveraging the capabilities of deep learning, we present an innovative solution that automates the classification process, providing farmers with a reliable and efficient tool for pest identification. Throughout this notebook, we will embark on a journey of data preprocessing, augmentation, and fine-tuning of ResNet and InceptionV3 models. We will explore transfer learning techniques to harness the knowledge of these state-of-the-art models while adapting them to our specific agricultural pest classification task. Furthermore, we will conduct thorough model evaluation, analyzing accuracy, loss, and generating insightful visualizations like confusion matrices to assess the performance of our models on different pest classes.

<p style="font-family: 'Arial', sans-serif; font-size: 18px; color: #333333;text-align: justify;">By the end of this notebook, we hope to showcase the effectiveness of deep learning models in agricultural pest classification and inspire the adoption of advanced technologies for precision agriculture. So, let's dive in and witness the power of ResNet and InceptionV3 in identifying and mitigating the challenges posed by agricultural pests.

#    Happy learning!

# Import required libraries

In [7]:
!pip install matplotlib-venn
!apt-get -qq install -y libfluidsynth1
# https://pypi.python.org/pypi/libarchive
!apt-get -qq install -y libarchive-dev && pip install -U libarchive
import libarchive
# https://pypi.python.org/pypi/pydot
!apt-get -qq install -y graphviz && pip install pydot
import pydot
!pip install cartopy
import cartopy

E: Package 'libfluidsynth1' has no installation candidate
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 67.4 MB/s eta 0:00:00


In [8]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
import itertools
import random
import warnings
warnings.filterwarnings("ignore")
# Import visualization libraries
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import cv2
import seaborn as sns

# Tensorflow Libraries
from tensorflow import keras
from tensorflow.keras import layers,models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import Callback, EarlyStopping,ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import MobileNetV2, VGG16
from tensorflow.keras import Model
from tensorflow.keras.layers.experimental import preprocessing

# System libraries
from pathlib import Path
import os.path

# Metrics
from sklearn.metrics import classification_report, confusion_matrix

sns.set(style='darkgrid')

ModuleNotFoundError: No module named 'tensorflow.keras.layers.experimental'

<p style="font-family: 'Arial', sans-serif; font-size: 18px; color: #333333;text-align: justify;">Setting the seed ensures that the random processes involved in training and testing models are initialized in the same state each time the code is executed. This helps in reducing randomness and making the results consistent across different runs.

In [ ]:
# Seed Everything to reproduce results for future use cases
def seed_everything(seed=42):
    # Seed value for TensorFlow
    tf.random.set_seed(seed)

    # Seed value for NumPy
    np.random.seed(seed)

    # Seed value for Python's random library
    random.seed(seed)

    # Force TensorFlow to use single thread
    # Multiple threads are a potential source of non-reproducible results.
    session_conf = tf.compat.v1.ConfigProto(
        intra_op_parallelism_threads=1,
        inter_op_parallelism_threads=1
    )

    # Make sure that TensorFlow uses a deterministic operation wherever possible
    tf.compat.v1.set_random_seed(seed)

    sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
    tf.compat.v1.keras.backend.set_session(sess)

In [ ]:
seed_everything()

<p style="font-family: 'Arial', sans-serif; font-size: 18px; color: #333333;text-align: justify;">Download Helper Functions and Import

# Data Preprocessing
<p style="font-family: 'Arial', sans-serif; font-size: 18px; color: #333333;text-align: justify;">Import the Dataset

In [ ]:
### We create a bunch of helpful functions throughout the course.
### Storing them here so they're easily accessible.

import tensorflow as tf

# Create a function to import an image and resize it to be able to be used with our model
def load_and_prep_image(filename, img_shape=224, scale=True):
  """
  Reads in an image from filename, turns it into a tensor and reshapes into
  (224, 224, 3).

  Parameters
  ----------
  filename (str): string filename of target image
  img_shape (int): size to resize target image to, default 224
  scale (bool): whether to scale pixel values to range(0, 1), default True
  """
  # Read in the image
  img = tf.io.read_file(filename)
  # Decode it into a tensor
  img = tf.image.decode_jpeg(img)
  # Resize the image
  img = tf.image.resize(img, [img_shape, img_shape])
  if scale:
    # Rescale the image (get all values between 0 and 1)
    return img/255.
  else:
    return img

# Note: The following confusion matrix code is a remix of Scikit-Learn's
# plot_confusion_matrix function - https://scikit-learn.org/stable/modules/generated/sklearn.metrics.plot_confusion_matrix.html
import itertools
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix

# Our function needs a different name to sklearn's plot_confusion_matrix
def make_confusion_matrix(y_true, y_pred, classes=None, figsize=(10, 10), text_size=15, norm=False, savefig=False):
  """Makes a labelled confusion matrix comparing predictions and ground truth labels.

  If classes is passed, confusion matrix will be labelled, if not, integer class values
  will be used.

  Args:
    y_true: Array of truth labels (must be same shape as y_pred).
    y_pred: Array of predicted labels (must be same shape as y_true).
    classes: Array of class labels (e.g. string form). If `None`, integer labels are used.
    figsize: Size of output figure (default=(10, 10)).
    text_size: Size of output figure text (default=15).
    norm: normalize values or not (default=False).
    savefig: save confusion matrix to file (default=False).

  Returns:
    A labelled confusion matrix plot comparing y_true and y_pred.

  Example usage:
    make_confusion_matrix(y_true=test_labels, # ground truth test labels
                          y_pred=y_preds, # predicted labels
                          classes=class_names, # array of class label names
                          figsize=(15, 15),
                          text_size=10)
  """
  # Create the confustion matrix
  cm = confusion_matrix(y_true, y_pred)
  cm_norm = cm.astype("float") / cm.sum(axis=1)[:, np.newaxis] # normalize it
  n_classes = cm.shape[0] # find the number of classes we're dealing with

  # Plot the figure and make it pretty
  fig, ax = plt.subplots(figsize=figsize)
  cax = ax.matshow(cm, cmap=plt.cm.Blues) # colors will represent how 'correct' a class is, darker == better
  fig.colorbar(cax)

  # Are there a list of classes?
  if classes:
    labels = classes
  else:
    labels = np.arange(cm.shape[0])

  # Label the axes
  ax.set(title="Confusion Matrix",
         xlabel="Predicted label",
         ylabel="True label",
         xticks=np.arange(n_classes), # create enough axis slots for each class
         yticks=np.arange(n_classes),
         xticklabels=labels, # axes will labeled with class names (if they exist) or ints
         yticklabels=labels)

  # Make x-axis labels appear on bottom
  ax.xaxis.set_label_position("bottom")
  ax.xaxis.tick_bottom()

  # Set the threshold for different colors
  threshold = (cm.max() + cm.min()) / 2.

  # Plot the text on each cell
  for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    if norm:
      plt.text(j, i, f"{cm[i, j]} ({cm_norm[i, j]*100:.1f}%)",
              horizontalalignment="center",
              color="white" if cm[i, j] > threshold else "black",
              size=text_size)
    else:
      plt.text(j, i, f"{cm[i, j]}",
              horizontalalignment="center",
              color="white" if cm[i, j] > threshold else "black",
              size=text_size)

  # Save the figure to the current working directory
  if savefig:
    fig.savefig("confusion_matrix.png")

# Make a function to predict on images and plot them (works with multi-class)
def pred_and_plot(model, filename, class_names):
  """
  Imports an image located at filename, makes a prediction on it with
  a trained model and plots the image with the predicted class as the title.
  """
  # Import the target image and preprocess it
  img = load_and_prep_image(filename)

  # Make a prediction
  pred = model.predict(tf.expand_dims(img, axis=0))

  # Get the predicted class
  if len(pred[0]) > 1: # check for multi-class
    pred_class = class_names[pred.argmax()] # if more than one output, take the max
  else:
    pred_class = class_names[int(tf.round(pred)[0][0])] # if only one output, round

  # Plot the image and predicted class
  plt.imshow(img)
  plt.title(f"Prediction: {pred_class}")
  plt.axis(False);

import datetime

def create_tensorboard_callback(dir_name, experiment_name):
  """
  Creates a TensorBoard callback instand to store log files.

  Stores log files with the filepath:
    "dir_name/experiment_name/current_datetime/"

  Args:
    dir_name: target directory to store TensorBoard log files
    experiment_name: name of experiment directory (e.g. efficientnet_model_1)
  """
  log_dir = dir_name + "/" + experiment_name + "/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
  tensorboard_callback = tf.keras.callbacks.TensorBoard(
      log_dir=log_dir
  )
  print(f"Saving TensorBoard log files to: {log_dir}")
  return tensorboard_callback

# Plot the validation and training data separately
import matplotlib.pyplot as plt

def plot_loss_curves(history):
  """
  Returns separate loss curves for training and validation metrics.

  Args:
    history: TensorFlow model History object (see: https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/History)
  """
  loss = history.history['loss']
  val_loss = history.history['val_loss']

  accuracy = history.history['accuracy']
  val_accuracy = history.history['val_accuracy']

  epochs = range(len(history.history['loss']))

  # Plot loss
  plt.plot(epochs, loss, label='training_loss')
  plt.plot(epochs, val_loss, label='val_loss')
  plt.title('Loss')
  plt.xlabel('Epochs')
  plt.legend()

  # Plot accuracy
  plt.figure()
  plt.plot(epochs, accuracy, label='training_accuracy')
  plt.plot(epochs, val_accuracy, label='val_accuracy')
  plt.title('Accuracy')
  plt.xlabel('Epochs')
  plt.legend();

def compare_historys(original_history, new_history, initial_epochs=5):
    """
    Compares two TensorFlow model History objects.

    Args:
      original_history: History object from original model (before new_history)
      new_history: History object from continued model training (after original_history)
      initial_epochs: Number of epochs in original_history (new_history plot starts from here)
    """

    # Get original history measurements
    acc = original_history.history["accuracy"]
    loss = original_history.history["loss"]

    val_acc = original_history.history["val_accuracy"]
    val_loss = original_history.history["val_loss"]

    # Combine original history with new history
    total_acc = acc + new_history.history["accuracy"]
    total_loss = loss + new_history.history["loss"]

    total_val_acc = val_acc + new_history.history["val_accuracy"]
    total_val_loss = val_loss + new_history.history["val_loss"]

    # Make plots
    plt.figure(figsize=(8, 8))
    plt.subplot(2, 1, 1)
    plt.plot(total_acc, label='Training Accuracy')
    plt.plot(total_val_acc, label='Validation Accuracy')
    plt.plot([initial_epochs-1, initial_epochs-1],
              plt.ylim(), label='Start Fine Tuning') # reshift plot around epochs
    plt.legend(loc='lower right')
    plt.title('Training and Validation Accuracy')

    plt.subplot(2, 1, 2)
    plt.plot(total_loss, label='Training Loss')
    plt.plot(total_val_loss, label='Validation Loss')
    plt.plot([initial_epochs-1, initial_epochs-1],
              plt.ylim(), label='Start Fine Tuning') # reshift plot around epochs
    plt.legend(loc='upper right')
    plt.title('Training and Validation Loss')
    plt.xlabel('epoch')
    plt.show()

# Create function to unzip a zipfile into current working directory
# (since we're going to be downloading and unzipping a few files)
import zipfile

def unzip_data(filename):
  """
  Unzips filename into the current working directory.

  Args:
    filename (str): a filepath to a target zip folder to be unzipped.
  """
  zip_ref = zipfile.ZipFile(filename, "r")
  zip_ref.extractall()
  zip_ref.close()

# Walk through an image classification directory and find out how many files (images)
# are in each subdirectory.
import os

def walk_through_dir(dir_path):
  """
  Walks through dir_path returning its contents.

  Args:
    dir_path (str): target directory

  Returns:
    A print out of:
      number of subdiretories in dir_path
      number of images (files) in each subdirectory
      name of each subdirectory
  """
  for dirpath, dirnames, filenames in os.walk(dir_path):
    print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")

# Function to evaluate: accuracy, precision, recall, f1-score
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
  """
  Calculates model accuracy, precision, recall and f1 score of a binary classification model.

  Args:
      y_true: true labels in the form of a 1D array
      y_pred: predicted labels in the form of a 1D array

  Returns a dictionary of accuracy, precision, recall, f1-score.
  """
  # Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  # Calculate model precision, recall and f1 score using "weighted average
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {"accuracy": model_accuracy,
                  "precision": model_precision,
                  "recall": model_recall,
                  "f1": model_f1}
  return model_results

In [ ]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

# Import series of helper functions for our notebook
from helper_functions import create_tensorboard_callback, plot_loss_curves, unzip_data, compare_historys, walk_through_dir, pred_and_plot

In [ ]:
dataset = "/kaggle/input/agricultural-pests-image-dataset"
walk_through_dir(dataset)

# Visualize the Pest Images

In [ ]:
from tensorflow.keras.preprocessing.image import load_img
# Step 2: Print 5 Images from Each Pest Subfolder

# Loop through each pest subfolder
for folder_name in os.listdir(dataset):
    folder_path = os.path.join(dataset, folder_name)

    # Check if the current item is a directory (pest subfolder)
    if os.path.isdir(folder_path):
        # Get a list of all images in the current pest subfolder
        images = [img for img in os.listdir(folder_path) if img.lower().endswith(('.png', '.jpg', '.jpeg'))]

        # Randomly select 5 images from the list
        selected_images = random.sample(images, 5)

        # Print the selected images in multiple columns
        fig, axes = plt.subplots(1, 5, figsize=(15, 3))
        fig.suptitle(folder_name)

        for i, image_name in enumerate(selected_images):
            image_path = os.path.join(folder_path, image_name)
            image = load_img(image_path, target_size=(224, 224))

            axes[i].imshow(image)
            axes[i].axis('off')

        plt.show()

<p style="font-family: 'Arial', sans-serif; font-size: 18px; color: #333333;text-align: justify;"> Create a dataframe of the pests with the image path and the labels

In [ ]:
# Step 2: Create DataFrame and Store Data

# Initialize lists to store image paths and corresponding labels
image_paths = []
labels = []

# Loop through each pest subfolder
for folder_name in os.listdir(dataset):
    folder_path = os.path.join(dataset, folder_name)

    # Check if the current item is a directory (pest subfolder)
    if os.path.isdir(folder_path):
        # Get a list of all images in the current pest subfolder
        images = [img for img in os.listdir(folder_path) if img.lower().endswith(('.png', '.jpg', '.jpeg'))]

        for image_name in images:
            image_path = os.path.join(folder_path, image_name)
            image_paths.append(image_path)
            labels.append(folder_name)

# Create a DataFrame from the collected data
data = {
    'image_path': image_paths,
    'label': labels
}
df = pd.DataFrame(data)

# Display the DataFrame
print(df)

<p style="font-family: 'Arial', sans-serif; font-size: 18px; color: #333333;text-align: justify;">Let us plot the distribution characteristics of the pests

In [ ]:
import seaborn as sns
# Plot the label distribution as a bar plot using Seaborn
plt.figure(figsize=(10, 6))
sns.set(style='whitegrid')  # Use a white grid background

# Generate the bar plot
sns.barplot(x=df['label'].value_counts().index, y=df['label'].value_counts().values)

# Customize the plot
plt.xlabel('Label', fontsize=14)
plt.ylabel('Count', fontsize=14)
plt.title('Pest Distribution', fontsize=16)
plt.xticks(rotation=45, ha='right')  # Rotate x-axis labels for better visibility
plt.tight_layout()

# Show the plot
plt.show()

#### It can be seen that last 4 pests have less images than others. The imbalance is not as dominant so we will leave it as it is.

# Train Test Splitting and Data Generator

In [ ]:
# Separate in train and test data
train_df, test_df = train_test_split(df, test_size=0.2, shuffle=True, random_state=42)

In [ ]:
train_generator = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.efficientnet_v2.preprocess_input,
    validation_split=0.2
)

test_generator = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.efficientnet_v2.preprocess_input
)

In [ ]:
# Split the data into three categories.
train_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='image_path',
    y_col='label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='training'
)

val_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='image_path',
    y_col='label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='validation'
)

test_images = test_generator.flow_from_dataframe(
    dataframe=test_df,
    x_col='image_path',
    y_col='label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=False
)

# Data Augmentation

In [ ]:
from tensorflow.keras.layers.experimental.preprocessing import RandomFlip, RandomRotation, RandomZoom, RandomContrast
from tensorflow.keras.models import Sequential

# Data Augmentation Step
augment = Sequential([
    RandomFlip("horizontal"),            # Random horizontal flipping
    RandomRotation(factor=0.2),          # Random rotation up to 20 degrees
    RandomZoom(height_factor=0.1, width_factor=0.1),  # Random zooming up to 10%
    RandomContrast(factor=0.1),          # Random contrast adjustment up to 10%
])

# Loading of a pretrained ResNet50 Model

In [ ]:

from tensorflow.keras.applications import ResNet50
resnet_model = ResNet50(
    include_top=False,
    weights='/kaggle/input/resnet50/tensorflow2/default/1/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5',
    input_shape=(224, 224, 3),
    pooling='max'
)

resnet_model.trainable = False

# Create checkpoint callback
checkpoint_path = "pests_cats_classification_model_checkpoint"
checkpoint_callback = ModelCheckpoint(checkpoint_path,
                                      save_weights_only=True,
                                      monitor="val_accuracy",
                                      save_best_only=True)

In [ ]:
# Setup EarlyStopping callback to stop training if model's val_loss doesn't improve for 5 epochs
early_stopping = EarlyStopping(monitor="val_loss",  # watch the validation loss metric
                               patience=5,         # stop training if no improvement for 5 epochs
                               restore_best_weights=True)  # restore best weights based on validation loss

# Add additional layers for the custom head of the model
inputs = resnet_model.input
x = augment(inputs)
x = tf.keras.layers.experimental.preprocessing.RandomFlip(mode='horizontal')(inputs)
x = tf.keras.layers.experimental.preprocessing.RandomRotation(0.2)(x)
x = resnet_model(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.45)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.45)(x)
outputs = Dense(12, activation='softmax')(x)

# Training the Model

In [ ]:
# Create the final model
model = Model(inputs=inputs, outputs=outputs)

# Compile the model
model.compile(
    optimizer=Adam(0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Train the model
history = model.fit(
    train_images,
    steps_per_epoch=len(train_images),
    validation_data=val_images,
    validation_steps=len(val_images),
    epochs=5,
    callbacks=[
        early_stopping,
        checkpoint_callback
    ]
)

<p style="font-family: 'Arial', sans-serif; font-size: 18px; color: #333333;text-align: justify;"> We can see the training has stopped early because there is no significant increase in the accuracy and it is wastefull to go for more epochs

In [ ]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_images, steps=len(test_images))

print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

<p style="font-family: 'Arial', sans-serif; font-size: 18px; color: #333333;text-align: justify;"> We have got an accuracy of 83 % which is not too bad.

# Model Evaluation

## Visualize the accuracy and loss plots

In [ ]:
# Plot training and validation loss over epochs
plt.figure(figsize=(10, 6))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch', fontsize=14)
plt.ylabel('Loss', fontsize=14)
plt.title('Training and Validation Loss', fontsize=16)
plt.legend()
plt.grid()
plt.show()

In [ ]:
# Plot training and validation accuracy over epochs
plt.figure(figsize=(10, 6))
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch', fontsize=14)
plt.ylabel('Accuracy', fontsize=14)
plt.title('Training and Validation Accuracy', fontsize=16)
plt.legend()
plt.grid()
plt.show()

<p style="font-family: 'Arial', sans-serif; font-size: 18px; color: #333333;text-align: justify;"> Here it can be seen that the model is performing better on validation data which indicates the model is not overfitting
    

# Model Predictions

In [ ]:
class_names = ['beetle', 'grasshopper', 'earthworms', 'ants', 'earwig', 'snail',
       'catterpillar', 'weevil', 'bees', 'moth', 'wasp', 'slug']  # Replace with your actual class names

# Make predictions on the test data
predictions = model.predict(test_images)

# Convert the predicted probabilities to class labels
predicted_labels = np.argmax(predictions, axis=1)

# Get the true labels from the test data generator
true_labels = test_images.classes

# Calculate the accuracy of the model on the test data
accuracy = np.mean(predicted_labels == true_labels)
print("Test Accuracy:", accuracy)

# Print some sample predictions along with true labels
num_samples = 10
print("Sample Predictions:")
for i in range(num_samples):
    print("True Label:", class_names[true_labels[i]], "| Predicted Label:", class_names[predicted_labels[i]])

### The predictions are very good

# Classification report and Confusion Matrix

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
class_names = ['beetle', 'grasshopper', 'earthworms', 'ants', 'earwig', 'snail','catterpillar', 'weevil']
# Get a batch of test data (you can adjust the batch size as needed)
batch_size = 10
test_data = next(iter(test_images))
images = test_data[0][:batch_size]  # Get the images from the test data batch
true_labels = test_data[1][:batch_size]  # Get the true labels from the test data batch

# Make predictions on the test data
predictions = model.predict(images)
predicted_labels = np.argmax(predictions, axis=1)

# Convert the one-hot encoded true_labels to class indices
true_labels = np.argmax(true_labels, axis=1)

# Generate the confusion matrix
cm = confusion_matrix(true_labels, predicted_labels)

# Plot the confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
plt.xlabel("Predicted Labels")
plt.ylabel("True Labels")
plt.title("Confusion Matrix")
plt.show()

# Generate the classification report
print("Classification Report:")
print(classification_report(true_labels, predicted_labels, target_names=class_names))


# Loading of a pretrained InceptionV3 Model

In [ ]:
from tensorflow.keras.applications import InceptionV3

# Load the pretained model
pretrained_model = InceptionV3(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet',
    pooling='max'
)

pretrained_model.trainable = True

# Create checkpoint callback
checkpoint_path = "pests_cats_classification_model_checkpoint"
checkpoint_callback = ModelCheckpoint(checkpoint_path,
                                      save_weights_only=True,
                                      monitor="val_accuracy",
                                      save_best_only=True)

# Setup EarlyStopping callback to stop training if model's val_loss doesn't improve for 5 epochs
early_stopping = EarlyStopping(monitor="val_loss",  # watch the validation loss metric
                               patience=5,         # stop training if no improvement for 5 epochs
                               restore_best_weights=True)  # restore best weights based on validation loss

# Add additional layers for the custom head of the model
inputs = tf.keras.Input(shape=(224, 224, 3))
x = augment(inputs)
x = tf.keras.layers.experimental.preprocessing.RandomFlip(mode='horizontal')(x)
x = tf.keras.layers.experimental.preprocessing.RandomRotation(0.2)(x)
x = pretrained_model(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.45)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.45)(x)
outputs = Dense(12, activation='softmax')(x)

# IneptionV3 Model Training


In [ ]:

# Create the final model
model = Model(inputs=inputs, outputs=outputs)

# Compile the model
model.compile(
    optimizer=Adam(0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Train the model
history = model.fit(
    train_images,
    steps_per_epoch=len(train_images),
    validation_data=val_images,
    validation_steps=len(val_images),
    epochs=100,
    callbacks=[
        early_stopping,
        checkpoint_callback
    ]
)


In [ ]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_images, steps=len(test_images))

print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

<p style="font-family: 'Arial', sans-serif; font-size: 18px; color: #333333;text-align: justify;"> The accuracy has been increased from 82 to 86 % with respect to InceptionV3 model

In [ ]:
# Plot training and validation loss over epochs
plt.figure(figsize=(10, 6))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch', fontsize=14)
plt.ylabel('Loss', fontsize=14)
plt.title('Training and Validation Loss', fontsize=16)
plt.legend()
plt.grid()
plt.show()

In [ ]:
# Plot training and validation accuracy over epochs
plt.figure(figsize=(10, 6))
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch', fontsize=14)
plt.ylabel('Accuracy', fontsize=14)
plt.title('Training and Validation Accuracy', fontsize=16)
plt.legend()
plt.grid()
plt.show()

<p style="font-family: 'Arial', sans-serif; font-size: 18px; color: #333333;text-align: justify;"> After 6 epochs, the loss is increasing and the accuracy of the validation set is decreasing. That is where earlystopping comes into play.

# InceptionV3 model prediction

In [ ]:
class_names = ['beetle', 'grasshopper', 'earthworms', 'ants', 'earwig', 'snail',
       'catterpillar', 'weevil', 'bees', 'moth', 'wasp', 'slug']  # Replace with your actual class names

# Make predictions on the test data
predictions = model.predict(test_images)

# Convert the predicted probabilities to class labels
predicted_labels = np.argmax(predictions, axis=1)

# Get the true labels from the test data generator
true_labels = test_images.classes

# Calculate the accuracy of the model on the test data
accuracy = np.mean(predicted_labels == true_labels)
print("Test Accuracy:", accuracy)

# Print some sample predictions along with true labels
num_samples = 10
print("Sample Predictions:")
for i in range(num_samples):
    print("True Label:", class_names[true_labels[i]], "| Predicted Label:", class_names[predicted_labels[i]])

# Conclusion
<p style="font-family: 'Arial', sans-serif; font-size: 18px; color: #333333;text-align: justify;">In this Kaggle notebook, we delved into the exciting domain of agricultural pest classification using deep learning techniques. Our main goal was to assist farmers in effectively identifying and managing common agricultural pests, which can significantly impact crop yield and quality.

<p style="font-family: 'Arial', sans-serif; font-size: 18px; color: #333333;text-align: justify;">Throughout the project, we employed two powerful pre-trained deep learning models, ResNet and InceptionV3, to tackle the pest classification task. After thorough data preprocessing and augmentation, we fine-tuned both models using transfer learning techniques, leveraging their pre-trained knowledge for our specific agricultural pest classification.

<p style="font-family: 'Arial', sans-serif; font-size: 18px; color: #333333;text-align: justify;">Upon evaluation, we observed that the InceptionV3 model outperformed ResNet, achieving an impressive accuracy of 86%. This notable improvement from 82% demonstrated the superiority of the InceptionV3 architecture for our classification task. By automating the pest identification process, our deep learning models offer a reliable and efficient tool for farmers to identify and manage agricultural pests. This technology can significantly contribute to sustainable agriculture practices, ensuring timely pest detection and precise management.

<p style="font-family: 'Arial', sans-serif; font-size: 18px; color: #333333;text-align: justify;">In conclusion, this project showcases the potential of deep learning in addressing real-world challenges faced by the agricultural sector. With continued research and innovation, we can harness the power of AI to enhance precision agriculture and contribute to global food security.

<p style="font-family: 'Arial', sans-serif; font-size: 18px; color: #333333;text-align: justify;">Let's continue exploring the realms of artificial intelligence and its applications in various fields, driving positive change and advancements for a better tomorrow. Happy learning and happy farming!